In [20]:
!pip install keras-tuner

In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [24]:
fashion_mnist=keras.datasets.fashion_mnist

In [25]:
(train_images, train_labels), (test_images, test_labels)=fashion_mnist.load_data()

In [26]:
train_images=train_images/255.0
test_images=test_images/255.0

In [27]:
train_images[0].shape

(28, 28)

In [28]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [49]:
def build_mode(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                                kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                                activation='relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                                kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                                activation='relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                                activation='relu'
                            ),
                            keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [50]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [51]:
tuner_search=RandomSearch(build_mode,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [52]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 38m 12s]
val_accuracy: 0.9171666502952576

Best val_accuracy So Far: 0.9171666502952576
Total elapsed time: 01h 28m 51s
INFO:tensorflow:Oracle triggered exit


In [53]:
model=tuner_search.get_best_models(num_models=1)[0]

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        179264    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3965056   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 4,146,730
Trainable params: 4,146,730
Non-trainable params: 0
_________________________________________________________________
